In [1]:
from google.colab import files

uploaded = files.upload()


Saving cloudy.csv to cloudy.csv
Saving rainy.csv to rainy.csv
Saving snowy.csv to snowy.csv
Saving sunny_open.csv to sunny_open.csv
Saving sunny_shade.csv to sunny_shade.csv


In [0]:
import numpy as np
import pandas as pd
import sklearn
%matplotlib inline
from sklearn.model_selection import cross_val_score
import matplotlib.pyplot as plt
#load data
cloudy =  pd.read_csv('cloudy.csv',header=0)
sunny_open = pd.read_csv('sunny_open.csv',header=0)
rainy = pd.read_csv('rainy.csv',header=0)
sunny_shade= pd.read_csv('sunny_shade.csv',header=0)
snowy= pd.read_csv('snowy.csv',header=0)

# delete NaN rows
cloudy=cloudy.loc[np.arange(1,cloudy.shape[0],2)].reset_index(drop=True)
sunny_open=sunny_open.loc[np.arange(1,sunny_open.shape[0],2)].reset_index(drop=True)
rainy=rainy.loc[np.arange(1,rainy.shape[0],2)].reset_index(drop=True)
sunny_shade=sunny_shade.loc[np.arange(1,sunny_shade.shape[0],2)].reset_index(drop=True)
snowy=snowy.loc[np.arange(1,snowy.shape[0],2)].reset_index(drop=True)

In [38]:
# X_train, y_train
from sklearn.model_selection import train_test_split
labels=[len(cloudy),len(sunny_open),len(rainy),len(sunny_shade),len(snowy)]
X = np.concatenate([cloudy,sunny_open,rainy,sunny_shade,snowy], axis=0)
X = X[:,0:14]
n = len(X)
y = np.zeros((n,1))
for i in range(len(X)):
  if i<labels[0]:
    y[i]=0
  elif i<labels[0]+labels[1]:
    y[i]=1
  elif i<labels[0]+labels[1]+labels[2]:
    y[i]=2
  elif i<labels[0]+labels[1]+labels[2]+labels[3]:
    y[i]=3
  else:
    y[i]=4
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print('training data:',len(X_train))
print('test data:',len(X_test))

training data: 930
test data: 233


In [39]:
# normalization
from sklearn import preprocessing
X = preprocessing.scale(X)
X_train = preprocessing.scale(X_train)
X_test = preprocessing.scale(X_test)

/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/validation.py:595: DataConversionWarning: Data with input dtype object was converted to float64 by the scale function.
  warnings.warn(msg, DataConversionWarning)


In [0]:
def tanh(x):
    x = np.array(x,dtype=np.float32)
    return np.tanh(x)

def tanh_derivative(x):
    return 1.0 - np.square(x)

def softmax(A):  
    expA = np.exp(A)
    return expA / expA.sum(axis=1, keepdims=True)

In [0]:
def one_hot_label(y):
    n = len(y)
    one_hot_y = np.zeros((n,5))
    for i in range(n):
        one_hot_y[i, int(y[i])] = 1
    return one_hot_y
y_train=one_hot_label(y_train)
y_test=one_hot_label(y_test)

In [0]:
class NeuralNetwork:
    def __init__(self, x, y,size):
        self.input      = x
        self.weights1   =  np.random.rand(self.input.shape[1],size)
        self.weights2   =  np.random.rand(size,5) 
        self.b1   = np.zeros((1,size)) 
        self.b2   = np.zeros((1,5))
        self.y          = y
        self.output     = np.zeros(self.y.shape)
        self.n          = len(x)

    def feedforward(self):
        self.layer1 = tanh(np.dot(self.input, self.weights1)+self.b1)
        self.output = softmax(np.dot(self.layer1, self.weights2)+self.b2) # n*5

    def backprop(self,lr,reg):
        # application of the chain rule to find derivative of the loss function with respect to weights2 and weights1
        delta3 = self.output-self.y
        delta2 = tanh_derivative(self.layer1)*np.dot(delta3, self.weights2.T)
        d_weights2 = np.dot(self.layer1.T, delta3)+reg*self.weights2
        d_b2 = np.sum(delta3, axis=0, keepdims=True)
        d_weights1 = np.dot(self.input.T, delta2)+reg*self.weights1
        d_b1 = np.sum(delta2, axis=0)

        # update the weights with the derivative (slope) of the loss function
        self.weights1 = self.weights1-lr * d_weights1
        self.b1 = self.b1-lr * d_b1
        self.weights2 = self.weights2-lr * d_weights2
        self.b2 = self.b2- lr * d_b2
    
    def train(self,iteration,lr,reg):
        for i in range(iteration):
            nn.feedforward()
            nn.backprop(lr,reg)
            #acc = nn.accuracy(nn.predict(self.input),self.y)
            #print('ite:',i,'acc:',acc)
    
    def predict(self,x):
        layer1 = tanh(np.dot(x, self.weights1)+self.b1)
        output = softmax(np.dot(layer1, self.weights2)+self.b2)
        predict = np.argmax(output, axis=1)
        return predict
    
    def accuracy(self,predict,y):
        count = 0
        y = np.argmax(y,axis=1)
        for i in range(len(y)):
            if predict[i]==y[i]:
                count+=1
        return count/len(y)

In [45]:
# cross validation
from sklearn.model_selection import KFold
n_splits = 5
kf = KFold(n_splits)
size = [10,20,50]
for i in range(3): 
    acc = 0
    for train_indices, valid_indices in kf.split(X_train):
      nn = NeuralNetwork(X_train[train_indices,:],y_train[train_indices],size=size[i])
      nn.train(iteration=100,lr=1e-3,reg=0.1)
      predict = nn.predict(X_train[valid_indices,:])
      #print(predict)
      acc += nn.accuracy(predict,y_train[valid_indices])
    print('size:',size[i],'accuracy:',acc/n_splits)

size: 10 accuracy: 0.9870967741935484
size: 20 accuracy: 0.9881720430107528
size: 50 accuracy: 0.9838709677419356


In [47]:
lr = [1e-4,5e-4,1e-3,5e-3]
for i in range(4): 
    acc = 0
    for train_indices, valid_indices in kf.split(X_train):
      nn = NeuralNetwork(X_train[train_indices,:],y_train[train_indices],size=10)
      nn.train(iteration=100,lr=lr[i],reg=0.1)
      predict = nn.predict(X_train[valid_indices,:])
      #print(predict)
      acc += nn.accuracy(predict,y_train[valid_indices])
    print('learning rate:',lr[i],'accuracy:',acc/n_splits)

learning rate: 0.0001 accuracy: 0.7623655913978495
learning rate: 0.0005 accuracy: 0.9483870967741936
learning rate: 0.001 accuracy: 0.9731182795698924
learning rate: 0.005 accuracy: 0.9903225806451612


In [49]:
reg = [1e-3,5e-3,1e-2,5e-2,1e-1]
for i in range(4): 
    acc = 0
    for train_indices, valid_indices in kf.split(X_train):
      nn = NeuralNetwork(X_train[train_indices,:],y_train[train_indices],size=20)
      nn.train(iteration=100,lr=5e-3,reg=reg[i])
      predict = nn.predict(X_train[valid_indices,:])
      acc += nn.accuracy(predict,y_train[valid_indices])
    print('regularization:',reg[i],'accuracy:',acc/n_splits)

regularization: 0.001 accuracy: 0.9935483870967742
regularization: 0.005 accuracy: 0.9913978494623656
regularization: 0.01 accuracy: 0.9913978494623656
regularization: 0.05 accuracy: 0.9935483870967742


Using test data

In [66]:
from sklearn.model_selection import KFold
from sklearn.utils import shuffle
n_splits = 5
kf = KFold(n_splits)
X,y = shuffle(X,y, random_state=0)
print('-----------Initial Accuracy-----------')
for train_indices, test_indices in kf.split(X):
  nn = NeuralNetwork(X[train_indices],y[train_indices],size=20)
  nn.train(iteration=1,lr=5e-3,reg=5e-2)
  predict = nn.predict(X[test_indices])
  acc = nn.accuracy(predict,y[test_indices])
  print('acc:',acc)

-----------Initial Accuracy-----------
acc: 0.004291845493562232
acc: 0.034334763948497854
acc: 0.13733905579399142
acc: 0.02586206896551724
acc: 0.0


In [80]:
from sklearn.model_selection import KFold
n_splits = 5
kf = KFold(n_splits)
print('-----------Accuracy-----------')
for train_indices, test_indices in kf.split(X):
  nn = NeuralNetwork(X[train_indices],y[train_indices],size=20)
  nn.train(iteration=100,lr=5e-3,reg=5e-2)
  predict = nn.predict(X[test_indices])
  acc = nn.accuracy(predict,y[test_indices])
  print('acc:',acc)

-----------Accuracy-----------
acc: 1.0
acc: 1.0


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: RuntimeWarning: overflow encountered in exp
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:10: RuntimeWarning: invalid value encountered in true_divide
  # Remove the CWD from sys.path while we load stuff.


acc: 1.0
acc: 1.0
acc: 1.0


In [0]:
# predict = nn.predict(X_test)
# print('predict:\n',predict.reshape(1,len(predict)),'\n')
# print('y_test:\n',y_test.reshape(1,len(y_test)))

Logistic regression

In [0]:
# X_train, y_train
from sklearn.model_selection import train_test_split
labels=[len(cloudy),len(sunny_open),len(rainy),len(sunny_shade),len(snowy)]
X = np.concatenate([cloudy,sunny_open,rainy,sunny_shade], axis=0)
X = X[:,0:14]
n = len(X)
y = np.zeros((n,1))
for i in range(len(X)):
  if i<labels[0]:
    y[i]=0
  elif i<labels[0]+labels[1]:
    y[i]=1
  elif i<labels[0]+labels[1]+labels[2]:
    y[i]=2
  elif i<labels[0]+labels[1]+labels[2]+labels[3]:
    y[i]=3
  else:
    y[i]=4

In [0]:
def LogisticRegression_GD(X_train, y_train, learning_rate):
    D = len(X_train[0])
    N = len(y_train) 
    C = 5
    y = np.zeros((N,C))
    X = np.hstack((X_train,np.ones((N,1))))
    eps = 1.0e-4
    for i in range(N):
        c = int(y_train[i,0])
        y[i,c] = 1
    W = np.random.rand(D+1,C)
    p = np.zeros((len(y_train),C))
    for t in range(10000):
        z=np.dot(X,W)
        max_z=(np.amax(z,axis=1).reshape(N,1))*np.ones((1,C))
        z=z-max_z
        z=np.array(z,dtype=np.float32)
        for j in range(C):
            p[:,j] = np.exp(z[:,j])/np.sum(np.exp(z),axis=1)
        W_gradient = - (np.dot(X.T,(y - p)))/N+0.5*W
        W = W - (learning_rate * W_gradient)
        if np.amax(learning_rate * W_gradient)<eps:
            break
        predict=np.argmax(p,axis=1) 
   
             
    return W[0:D,:], W[D,:],predict

In [0]:
def precision(y_p,y_t):
    error = 0
    for k in range(len(y_t)):
        if y_p[k]!=y_t[k]:
            error = error + 1
    return 1-float(error/len(y_t))

In [22]:
# normalization
from sklearn import preprocessing
X = preprocessing.scale(X)

# k fold cross validation
print('------k fold cross validation------')
kf = KFold(n_splits=5, random_state=42, shuffle=True)
for train, test in kf.split(X):
  W, b, predict= LogisticRegression_GD(X[train], y[train], 0.01)
  #print ('training precision =', precision(predict,y_train))
  p = np.zeros((len(y[test]),4))
  zz = np.dot(X[test],W)
  zz = np.array(zz,dtype=np.float32)
  zz = np.exp(zz)
  z_sum = np.sum(zz,axis=1)
  for j in range(4):
    z = np.dot(X[test],W[:,j])
    z = np.array(z,dtype=np.float32)
    p[:,j] = np.exp(z)/z_sum
  predict = np.argmax(p,axis=1)
  print ('test precision =', precision(predict,y[test]))
   

------k fold cross validation------
test precision = 0.8243902439024391
test precision = 0.7609756097560976
test precision = 0.7658536585365854
test precision = 0.7658536585365854
test precision = 0.7303921568627452


In [0]:
# print('predict:\n',predict.reshape(1,len(predict)),'\n')
# print('y_test:\n',y_test.reshape(1,len(y_test)))